<a href="https://colab.research.google.com/github/nmalby/APS/blob/main/APS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#APS Explorer

##Configuration

#####Template

In [ ]:
'''
Parameters
  Configuration
    Shell
      Loader      - Belt, 1, 2, 3, 4, 6, 7, 8             - Multiple            CheckBox
      Modules
        Head      - AP, Heavy, AP, Thump, Chem, Special   - Multiple            Checkbox
        Fuse      - None, Single, Double                  - Single              Radio
        Base      - None, BB, Trace                       - Single              Radio
    Gun
      Gauge       - 18-500                                - Single              Field/Slider
      Barrels     - 1-6                                   - Single              Radio
      Evacuator   - Y/N                                   - Single              Radio
      Ejector     - Y/N                                   - Single              Radio
      Rail        - Y/N                                   - Single              Radio
      Rail Limit  - 1000-20000                            - Single              Field/Slider
  
    Engine
      PPM         - 200-800                               - Single              Field/Slider
      PPV         - 10-200                                - Single              Field/Slider
      PPB         - 10-200                                - Single              Field/Slider

  Requirements
    AP            - 0-60                                  - Single              Field/Slider
    Velocity      - 500-2000                              - Multiple (min-max)  Slider
    (Pen Req)     -                                       - Single              Field

  Goal Weights
    KD            - 0-1                                   - Single              Field/Slider
    CD            - 0-1                                   - Single              Field/Slider
    ROF           - 0-1                                   - Single              Field/Slider
    (Uptime)      - 0-1 (Belt)                            - Single              Field/Slider
    Volume        - 0-1                                   - Single              Field/Slider
    Cost          - 0-1 (Build +10 min)                   - Single              Field/Slider
'''

###Shell

In [24]:
# Parameters
# TODO: Add Form

# loaderChoices = []
# headChoices = []
# fuseChoice = 0
# baseChoice = 0

# loaderLengths = ["Belt", 1, 2, 3, 4, 6, 8]

shellParams = {
  'loaderChoices' : {
    'type': 'MultipleChoice',
    'options': ["Belt", 1, 2, 3, 4, 6, 8],
    'value': [4],
    'description': 'Loader lengths',
    'disabled': False
  },
  'headChoices' : {
    'type': 'MultipleChoice',
    'options': headData.keys(),
    'value': ['AP'],
    'description': 'Heads'
  },
  'fuseCount': {
    'type': 'IntSlider',
    'min': 0,
    'max': 5,
    'step': 1,
    'value': 0,
    'description': 'Fuses'
  },
  'baseChoices' : {
    'type': 'MultipleChoice',
    'options': baseData.keys(),
    'value': [],
    'description': 'Bases'
  }
}

initWidgets(shellParams)

AttributeError: ignored

###Gun

In [13]:
# gaugeChoice = 0

# evacChoice = True
# ejecChoice = True

# barrels = 1
# clipsPerLoader = 1

# railChoice = True
# railLimit = 5000

gunParams = {
  'gaugeChoice': {
    'type': 'BoundedIntText_IntSlider',
    'min': 18,
    'max': 500,
    'step': 1,
    'value': 250,
    'description': 'Gauge'
  },
  'evacChoice': {
    'type': 'Checkbox',
    'description': 'Evacuators'
  },
  'ejecChoice': {
    'type': 'Checkbox',
    'description': 'Ejectors'
  },
  'barrelsChoice': {
    'type': 'BoundedIntText_IntSlider',
    'min': 1,
    'max': 6,
    'step': 1,
    'value': 1,
    'description': 'Barrels'
  },
  'clipsPerLoader': {
    'type': 'BoundedIntText_IntSlider',
    'min': 1,
    'max': 6,
    'step': 1,
    'value': 1,
    'description': 'Clips per loader'
  },
  'railChoice': {
    'type': 'Checkbox',
    'description': 'Railgun'
  },
  'railLimit': {
    'type': 'BoundedIntText_IntSlider',
    'min': 1000,
    'max': 20000,
    'step': 5000,
    'value': 1000,
    'description': 'Rail draw limit'
  }
}



In [19]:
import ipywidgets as widgets
from IPython.display import display
import datetime

# Sample dictionary to load options for the multiple-choice checkbox
options_dict = {'Option 1': 1, 'Option 2': 2, 'Option 3': 3}

# Dictionary to store parameters and their respective widget configurations
params_config = {
  'param1': {'type': 'BoundedIntText_IntSlider', 'min': 0, 'max': 10, 'step': 1},
  'param2': {'type': 'IntRangeSlider', 'min': 0, 'max': 10, 'step': 1},
  'param3': {'type': 'RadioButtons', 'options': ['Option A', 'Option B', 'Option C']},
  'param4': {'type': 'MultipleChoice', 'options': options_dict},
  'param5': {'type': 'Checkbox'}
}

initWidgets(params_config)

# Function to act as an event handler
def paramsChanged(change):
  return datetime.datetime.now()

# Iterate through the dictionary
def initWidgets(params_config):
  for param, config in params_config.items():
    param_type = config['type']
    
    if param_type == 'BoundedIntText_IntSlider':
      int_text = widgets.BoundedIntText(description=param, min=config['min'], max=config['max'], step=config['step'])
      int_slider = widgets.IntSlider(min=config['min'], max=config['max'], step=config['step'])
      
      widgets.jslink((int_text, 'value'), (int_slider, 'value'))
      
      int_text.observe(paramsChanged, names='value')
      int_slider.observe(paramsChanged, names='value')
      
      display(int_text, int_slider)

    elif param_type == 'IntRangeSlider':
      int_range_slider = widgets.IntRangeSlider(min=config['min'], max=config['max'], step=config['step'])
      
      int_range_slider.observe(paramsChanged, names='value')
      
      display(int_range_slider)

    elif param_type == 'MultipleChoice':
      hbox = widgets.HBox([widgets.Label(param)])
      for option, value in config['options'].items():
        checkbox = widgets.Checkbox(description=option)
        checkbox.metadata = {'value': value}
        checkbox.observe(paramsChanged, names='value')
        hbox.children += (checkbox,)

      display(hbox)
    elif param_type == 'RadioButtons':
      radio_buttons = widgets.RadioButtons(description=param, options=config['options'])
      
      radio_buttons.observe(paramsChanged, names='value')
      
      display(radio_buttons)
    elif param_type == 'Checkbox':
      checkbox = widgets.Checkbox(description=param)
      checkbox.observe(paramsChanged, names='value')
      display(checkbox)

BoundedIntText(value=0, description='param1', max=10)

IntSlider(value=0, max=10)

IntRangeSlider(value=(2, 7), max=10)

RadioButtons(description='param3', options=('Option A', 'Option B', 'Option C'), value='Option A')

Checkbox(value=False, description='param5')

###Engine

In [ ]:
PPM = 500
PPV = 40

PPB = 0

###Goal

In [ ]:
# Goal Weights
# TODO: Add Form
KDWeight = 1
CDWeight = 0
ROFWeight = .5
BeltUptimeWeight = 0
VolumeWeight = .5
CostWeight = .5

##Implementation

In [ ]:
paramsChanged =

In [ ]:
for loader in loaderChoices:
  for head in headChoices:
    template = {
      "loader" : loader,
      "guage" : gaugeChoice,
      "head" : head,
    
    }
    
    # TODO: create shell object
    #       run goal function

###APS Formulas

In [ ]:
class Shell:
  def __init__(self, template):
    self.gauge = template['gauge']
    self.loader = template['loader']
    self.head = template['head']
    self.body = template['body']
    self.fuse = template['fuse']
    self.base = template["base"]
    self.gp = template['gp']
    self.rc = template['rc']
    self.railCoef = template['railCoef'] if template['railCoef'] else 0
    self.railLimit = template['railLimit']      # user restriction

    self.gaugeCoefficient = (self.gauge**3 / 125000000.0) ** 0.6


  def loaderLength(self):
    return self.loader * 1000 if template["loader"] != "Belt" else 1000

  def headLength(self):
    hd = headData[self.head]
    return min(self.gauge, hd.maxLength or 500);

  def railMax(self):
    return min(self.railLimit, self.gaugeCoefficient * (self.effectiveProjectileModuleCount() + 0.5 * (self.rc or 0)))

  def railDraw(self):
    return 1

  def velocity(self):
    return 1

  def kineticDamage(self):
    return 1
  
  def AP(self):
    AP = self.velocity * self.overallArmorPierceModifier() * 0.0175;
    return AP
  
  def effectiveKineticDamage(self):
    kd = self.kineticDamage
    ap = self.AP
    ekd =  kd * ap
    return ekd
  
  def reloadTime(self):
    reload  = (self.gauge ** 3 / 125000000)** 0.45 * (2 + self.effectiveProjectileModuleCount() + 0.25 * (self.gp + (self.rc or 0))) * 17.5
    
    return 0.75 * reload * (self.gauge / 1000) ** 0.45 if self.loader == "Belt" else reload

  def effectiveProjectileModuleCount(self):
    return 1
  
  def overallArmorPierceModifier(self):
    return 1

  def loaderVolume(self):
    return 1
  
  def recoilVolume(self):
    return 1
  
  def coolerVolume(self):
    return 1
  
  def chargerVolume(self):
    return 1
  
  def magnetVolume(self):
    return 1
  
  def engineVolume(self):
    return (self.maxDraw() * (self.railDraw or 0)) / self.reloadTime() / PPV
    
  def volumePerIntake(self):
    return max(
      self.loaderVolume() +
      self.recoilVolume() +
      self.coolerVolume() +
      self.chargerVolume() +
      self.magnetVolume() +
      self.engineVolume(),
      2
    )
  
  def loaderCost(self):
    return 1
  
  def recoilCost(self):
    return self.recoilVolume() * costData["Recoil"];
  
  def coolerCost(self):
    return self.coolerVolume() * costData["Cooler"];

  def chargerCost(self):
    return 1

  def magnetCost(self):
    return 1

  def engineCost(self):
    return 1
  

In [ ]:
def optimizationGoalSimple(shell):
  return (shell.KineticDamage()** clampWeight(KDWeight) * shell.ChemDamage()** clampWeight(CDWeight) * shell.Uptime()** clampWeight(BeltUptimeWeight)) / \
          shell.ReloadTime()** clampWeight(ROFWeight) / shell.VolumePerIntake()** clampWeight(VolumeWeight) / shell.CostPerIntake()** clampWeight(CostWeight)

In [ ]:
def optimizationGoalDetailed(shell):
  return (shell.KineticDamageVsAngled()** clampWeight(KDWeight) * shell.AppliedChemDamage()** clampWeight(CDWeight) * shell.Uptime()** clampWeight(BeltUptimeWeight)) / \
          shell.ReloadTime()** clampWeight(ROFWeight) / shell.VolumePerIntake()** clampWeight(VolumeWeight) / shell.CostPerIntake()** clampWeight(CostWeight)

In [ ]:
def firstPassKeys():
  keys = {"GP", "Solid", "Sabot"}

  if CDWeight > 0:
    keys.add("Chem")
  if railChoice:
    keys.add("RailDraw")
  
  return keys

In [ ]:
def clampWeight(num):
   return max(min(num, 1), 1e6)

###Shell Data

In [2]:
loaderLengths = ["Belt", 1, 2, 3, 4, 6, 8]

headData = {
  "AP": {
    "VelocityMod": 1.6,
    "KineticDamageMod": 1.0,
    "ArmorPierceMod": 1.65,
    "ChemMod": 1.0,
    "ShortName": "AP",
  },
  "Sabot": {
    "VelocityMod": 1.6,
    "KineticDamageMod": 0.85,
    "ArmorPierceMod": 2.5,
    "ChemMod": 0.25,
    "ShortName": "Sab",
  },
  "Heavy": {
    "VelocityMod": 1.45,
    "KineticDamageMod": 1.65,
    "ArmorPierceMod": 1.0,
    "ChemMod": 1.0,
    "ShortName": "Hvy",
  },
  "HP": {
    "VelocityMod": 1.45,
    "KineticDamageMod": 1.2,
    "ArmorPierceMod": 1.0,
    "ChemMod": 1.0,
    "ShortName": "HP",
  },
  "Chem": {
    "VelocityMod": 1.3,
    "KineticDamageMod": 1.0,
    "ArmorPierceMod": 0.1,
    "ChemMod": 1.0,
    "IsChem": True,
    "ShortName": "Chem",
  },
  "Special": {
    "VelocityMod": 1.45,
    "KineticDamageMod": 0.1,
    "ArmorPierceMod": 0.1,
    "ChemMod": 1.0,
    "IsChem": True,
    "ShortName": "Spc",
  },
}
bodyData = {
  "SolidBody": {
    "VelocityMod": 1.1,
    "KineticDamageMod": 1.0,
    "ArmorPierceMod": 1.0,
    "ChemMod": 1.0,
    "ShortName": "Sol",
  },
  "SabotBody": {
    "VelocityMod": 1.1,
    "KineticDamageMod": 0.8,
    "ArmorPierceMod": 1.4,
    "ChemMod": 0.25,
    "ShortName": "Sab",
  },
  "ChemBody": {
    "VelocityMod": 1.0,
    "KineticDamageMod": 1.0,
    "ArmorPierceMod": 0.1,
    "ChemMod": 1.0,
    "ShortName": "Chem",
  },
  "Fuse": {
    "VelocityMod": 1.0,
    "KineticDamageMod": 1.0,
    "ArmorPierceMod": 1.0,
    "ChemMod": 1.0,
    "MaxLength": 100,
    "ShortName": "Fuse",
  }
}
baseData = {
  "BaseBleeder": {
    "VelocityMod": 1.0,  # special +0.15 bonus handled elsewhere
    "KineticDamageMod": 1.0,
    "ArmorPierceMod": 1.0,
    "ChemMod": 1.0,
    "MaxLength": 100,
    "ShortName": "BB",
  },
  "Tracer": {
    "VelocityMod": 1.0,
    "KineticDamageMod": 1.0,
    "ArmorPierceMod": 1.0,
    "ChemMod": 1.0,
    "MaxLength": 100,
    "ShortName": "Tracer",
  }
}



###Cost Data

In [ ]:
costData = {
  "InputFeeder": 50,
  "Cooler": 50,
  "RecoilAbsorber": 80, # 80 per meter, longer ones scale linearly
  "RailCharger": 400,
  "RailMagnet": 300,
  "Loader": {
    "Belt": 600,
    1: 240,
    2: 300,
    3: 330,
    4: 360,
    5: 390,
    6: 420,
    7: 450,
    8: 480,
  },
  "Clip": {
    "Belt": 160,
    1: 160,
    2: 200,
    3: 220,
    4: 240,
    5: 260,
    6: 280,
    7: 300,
    8: 320,
  },
  "Ejector": 10,
}